This analyses the two corpuses PA and YT on the basis of association of words with eight emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). The [NRC lexicon](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) is used.


In [1]:
import docx
import gensim
import nltk
import re
import spacy
from scipy.stats import mannwhitneyu

NRC_EMOTION_LEXICON_PATH = "Input/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
nlp = spacy.load('en', disable=['parser', 'ner'])

Read given input document file and return list of lines

In [2]:
def read_input_file(file_path):
    doc = docx.Document(file_path)
    total_lines = list()
    for i in doc.paragraphs:
        total_lines.append(i.text)
    return total_lines
filename = 'Input/US3_ALL_TRANSCRIPTS.docx'
lines = read_input_file(filename)

This method groups the given text into two groups(PA and YT) and removes unnecessary lines and characters.

In [3]:
def group_to_corpuses(lines_in_input):
    yt_group = list()
    pa_group = list()
    initial_group_flag = None
    regex = re.compile('^P[0-9]+$')

    for line in lines_in_input:
        if not line.strip():
            continue
        if line.strip().startswith("Joni: ") or line.strip().startswith("Jim:"):
            continue
        if line.strip().startswith("R:") or line.strip().startswith("R: ") or line.strip().startswith("R :"):
            continue
        if "R:" in line:
            lines_list = line.splitlines()
            for single_line in lines_list:
                if single_line.startswith("R:"):
                    lines_list.remove(single_line)
            line = ",".join(lines_list)

        if re.match(regex, line):
            continue
        if line.strip().startswith('2018-11-') or line.strip().startswith('Total experiment talk time:'):
            continue
        line = re.sub(r'\[[^()]*\]', '', line)  # regex removing text between brackets

        line = line.replace('...', ' ,')
        line = line.replace('…', ' ,')

        if not line.strip().startswith("P:") and not line.strip().startswith('YT') and not line.strip().startswith(
                'PA'):
            line = "".join([i if ord(i) < 128 else ' ' for i in line])
            if line.strip().startswith("R :") or line.strip().startswith("R4.") or line.strip().startswith("P10"):
                continue

        if line.strip().lower() == 'yt':
            initial_group_flag = 'yt'
            continue
        elif line.strip().lower() == 'pa':
            initial_group_flag = 'pa'
            continue
        elif line:
            if initial_group_flag == 'pa':
                pa_group.append(line)
            else:
                yt_group.append(line)
    return pa_group, yt_group
pa_group, yt_group = group_to_corpuses(lines)

Tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether. Gensim’s simple_preprocess() is great for this. Additionally I have set deacc=True to remove the punctuations.

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))
pa_group = sent_to_words(pa_group)
yt_group = sent_to_words(yt_group)

Lemantization of the tokenized words and joining them into sentences.

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(" ".join(
            [token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
pa_group = lemmatization(pa_group)
pa_group = ". ".join(pa_group)
yt_group = lemmatization(yt_group)
yt_group = ". ".join(yt_group)

Splitter class use english pickle splitter before tokenization

In [6]:
class Splitter(object):
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        """
        input format: a paragraph of text
        output format: a list of lists of words.
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        """
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences
splitter = Splitter()

Parts of Speech tagging of splitted words in a sentence.

In [7]:
class POSTagger(object):
    def __init__(self):
        pass

    def pos_tag(self, sentences):
        """
        input format: list of lists of words
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        output format: list of lists of tagged tokens. Each tagged tokens has a
        form, a lemma, and a list of tags
            e.g: [[('this', 'this', ['DT']), ('is', 'be', ['VB']), ('a', 'a', ['DT']), ('sentence', 'sentence', ['NN'])],
                    [('this', 'this', ['DT']), ('is', 'be', ['VB']), ('another', 'another', ['DT']), ('one', 'one', ['CARD'])]]
        """

        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        # adapt format
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos
postagger = POSTagger()

Read the NRC emotion lexicon and tag the words with emotion and lexicon. Use this tagged data to find the emotions and sentiment of each word of the POS tagged PA and YT corpus sentences.

In [8]:
class DictionaryTagger(object):
    def __init__(self, file_path):
        self.dictionary = dict()
        self.max_key_size = 0
        with open(file_path, "r", encoding="utf-8") as nrc_file:
            for line in nrc_file.readlines():
                if not line.strip():
                    continue
                line = re.sub(r'\s+', '\t', line)
                splited = line.replace("\n", "").split("\t")
                word, emotion, value = splited[0], splited[1], splited[2]

                if word in self.dictionary.keys():
                    self.dictionary[word].append((emotion, int(value)))
                else:
                    self.dictionary[word] = [(emotion, int(value))]

    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        """
        the result is only one tagging of all the possible ones.
        The resulting tagging is determined by these two priority rules:
            - longest matches have higher priority
            - search is made from left to right
        """
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0:
            self.max_key_size = N
        i = 0
        while i < N:
            j = min(i + self.max_key_size, N)  # avoid overflow
            tagged = False
            while j > i:
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()
                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form
                if literal in self.dictionary:
                    # self.logger.debug("found: %s" % literal)
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token:  # if the tagged literal is a single token, conserve its previous taggings:
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1
            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence
    
dicttagger = DictionaryTagger(NRC_EMOTION_LEXICON_PATH)


In [9]:
splitted_pa_sentences = splitter.split(pa_group)

pos_tagged_pa_sentences = postagger.pos_tag(splitted_pa_sentences)

dict_tagged_pa_sentences = dicttagger.tag(pos_tagged_pa_sentences)

splitted_yt_sentences = splitter.split(yt_group)

pos_tagged_yt_sentences = postagger.pos_tag(splitted_yt_sentences)

dict_tagged_yt_sentences = dicttagger.tag(pos_tagged_yt_sentences)

Get sentiments and emotions for each word from dictionary tagged data.

In [10]:
def get_sentiment(sentiment):
    if not isinstance(sentiment, tuple):
        return dict()
    results = dict()
    results.update({sentiment[0]: sentiment[1]})
    return results

Aggregate the sentiment score and emotions

In [11]:
def sentiment_score(dict_tagged_sentences):
    emotions = dict()
    for sentence in dict_tagged_sentences:
        for token in sentence:
            for tag in token[2]:
                value = get_sentiment(tag)
                if not value:
                    continue
                emotions.update({tag[0]: emotions.get(tag[0], 0) + value.get(tag[0], 0)})
    return emotions

pa_sentiment = sentiment_score(dict_tagged_pa_sentences)
yt_sentiment = sentiment_score(dict_tagged_yt_sentences)

Perform MannWhitney-U test to compare the emotions and sentiments between PA corpus and YT corpus

In [12]:
def mann_whitney_u_test(group_pa, group_yt):
    print("PA Sentiment: %s" % group_pa)
    print("YT Sentiment: %s" % group_yt)
    print("Mann Whitney-u Test:")
    pa_count = list(group_pa.values())
    yt_count = list(group_yt.values())
    try:
        mw_stat, mw_p = mannwhitneyu(pa_count, yt_count)
    except ValueError:
        mw_stat = -1  # in case of ties, Mann-Whitney cannot rank, and so cannot calculate U
        mw_p = -1

    print("MannWhitney U Value: %s" % mw_stat)
    print("MannWhitney rho Value: %s" % mw_p)
mann_whitney_u_test(pa_sentiment, yt_sentiment)

PA Sentiment: {'anger': 57, 'anticipation': 353, 'disgust': 202, 'fear': 153, 'joy': 211, 'negative': 167, 'positive': 940, 'sadness': 235, 'surprise': 150, 'trust': 494}
YT Sentiment: {'anger': 55, 'anticipation': 461, 'disgust': 92, 'fear': 205, 'joy': 169, 'negative': 122, 'positive': 636, 'sadness': 93, 'surprise': 88, 'trust': 417}
Mann Whitney-u Test:
MannWhitney U Value: 38.0
MannWhitney rho Value: 0.19233653136775436
